In [121]:
model_number = '_model_2.4_'

<img src="files/model_1_outline.png">

# change log
### changing shuffle to False
### changing sequence rate 5 -> 20
### INCREASING GRU UNITS 64 -> 128

## NEW TO THIS EXP: 60

sequence generator shape (corrected) (b, n, w, h, c)

In [122]:
import os
import glob
import keras

In [123]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard
import tensorflow as tf
import datetime

In [124]:
# from patrice's blogpost
from keras_video import VideoFrameGenerator

In [125]:
classes = ['suture_throws', 'hand_ties', 'thread_cuts']
classes.sort()
print(classes)

['hand_ties', 'suture_throws', 'thread_cuts']


In [126]:
# some global params
SIZE = (100, 100) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 30 # num frames in sequence 
BS = 8 # Batch size

In [127]:
# pattern to get videos and classes
glob_pattern='../data_v3_model_2.2/model_data/{classname}/*.avi'

In [128]:
# # for data augmentation
# data_aug = keras.preprocessing.image.ImageDataGenerator(
#     zoom_range=.1,
#     horizontal_flip=True,
#     rotation_range=8,
#     width_shift_range=.2,
#     height_shift_range=.2)

In [129]:
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=False,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

class hand_ties, validation count: 121, train count: 247
class suture_throws, validation count: 122, train count: 248
class thread_cuts, validation count: 121, train count: 248
Total data: 3 classes for 743 files for train


In [130]:
# getting validation data
valid = train.get_validation_generator()

Total data: 3 classes for 364 files for validation


In [131]:
# import keras_video.utils
# keras_video.utils.show_sample(train)

In [132]:
#valid.files

## BUILD CONV NET

In [133]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
def build_mobilenet(shape=(224, 224, 3), nbout=3):
    model = keras.applications.mobilenet.MobileNet(
        include_top=False,
        input_shape=shape,
        weights='imagenet')
    # Keep 9 layers to train﻿﻿
    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
    output = GlobalMaxPool2D()
    return keras.Sequential([model, output])

## Build GRU

In [134]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout
# Shape (5, 112, 112, 3) 5 - time sequence length 112x112 = height vs width 3 - num channels
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    # KEY = allows you to add a time sequence to a layer one at a time
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(128))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

## this is where you tell the model how to train - loss function, weight update mechanism

In [135]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
# action model - GRU set up for Time shifted CNN
model = action_model(INSHAPE, len(classes))

# this is where you tell the model how to train - loss function, weight update mechanism
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

## epochs, call backs

In [136]:
log_dir = "logs/fit/" + model_number + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
print('logs for this run are here: {}'.format(log_dir))

logs for this run are here: logs/fit/_model_2.4_20210512-100145


In [ ]:
EPOCHS=100
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'model2_3_1_chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
    tensorboard_callback
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/100
92/92 [==============================] - 471s 5s/step - loss: 1.3037 - acc: 0.2826 - val_loss: 1.1083 - val_acc: 0.3250

Epoch 00001: saving model to model2_3_1_chkp\weights.01-1.11.hdf5
Epoch 2/100
92/92 [==============================] - 135s 1s/step - loss: 1.1428 - acc: 0.3393 - val_loss: 1.1064 - val_acc: 0.3389

Epoch 00002: saving model to model2_3_1_chkp\weights.02-1.11.hdf5
Epoch 3/100
92/92 [==============================] - 138s 2s/step - loss: 1.1303 - acc: 0.2756 - val_loss: 1.1167 - val_acc: 0.3361

Epoch 00003: saving model to model2_3_1_chkp\weights.03-1.12.hdf5
Epoch 4/100
92/92 [==============================] - 138s 2s/step - loss: 1.1475 - acc: 0.3087 - val_loss: 1.0993 - val_acc: 0.3361

Epoch 00004: saving model to model2_3_1_chkp\weights.04-1.10.hdf5
Epoch 5/100
92/92 [==============================] - 137s 1s/step - loss: 1.1068 - acc: 0.3609 - val_loss: 1.1098 - val_acc: 0.3361

Epoch 00005: saving model to model2_3_1_chkp\weights.05-1.11.hdf5
Epoch

Epoch 41/100
92/92 [==============================] - 165s 2s/step - loss: 1.0983 - acc: 0.2886 - val_loss: 1.0985 - val_acc: 0.3389

Epoch 00041: saving model to model2_3_1_chkp\weights.41-1.10.hdf5
Epoch 42/100
92/92 [==============================] - 174s 2s/step - loss: 1.0982 - acc: 0.3242 - val_loss: 1.0985 - val_acc: 0.3389

Epoch 00042: saving model to model2_3_1_chkp\weights.42-1.10.hdf5
Epoch 43/100
92/92 [==============================] - 170s 2s/step - loss: 1.0989 - acc: 0.3514 - val_loss: 1.0985 - val_acc: 0.3389

Epoch 00043: saving model to model2_3_1_chkp\weights.43-1.10.hdf5
Epoch 44/100
92/92 [==============================] - 169s 2s/step - loss: 1.0994 - acc: 0.2747 - val_loss: 1.0985 - val_acc: 0.3389

Epoch 00044: saving model to model2_3_1_chkp\weights.44-1.10.hdf5
Epoch 45/100
92/92 [==============================] - 160s 2s/step - loss: 1.0987 - acc: 0.3033 - val_loss: 1.0985 - val_acc: 0.3389

Epoch 00045: saving model to model2_3_1_chkp\weights.45-1.10.hdf5


92/92 [==============================] - 157s 2s/step - loss: 1.0990 - acc: 0.3088 - val_loss: 1.0985 - val_acc: 0.3389

Epoch 00080: saving model to model2_3_1_chkp\weights.80-1.10.hdf5
Epoch 81/100
92/92 [==============================] - 159s 2s/step - loss: 1.0980 - acc: 0.3442 - val_loss: 1.0985 - val_acc: 0.3389

Epoch 00081: saving model to model2_3_1_chkp\weights.81-1.10.hdf5
Epoch 82/100
92/92 [==============================] - 177s 2s/step - loss: 1.0971 - acc: 0.4416 - val_loss: 1.0985 - val_acc: 0.3389

Epoch 00082: saving model to model2_3_1_chkp\weights.82-1.10.hdf5
Epoch 83/100
92/92 [==============================] - 170s 2s/step - loss: 1.0980 - acc: 0.3820 - val_loss: 1.0985 - val_acc: 0.3389

Epoch 00083: saving model to model2_3_1_chkp\weights.83-1.10.hdf5
Epoch 84/100
92/92 [==============================] - 179s 2s/step - loss: 1.0997 - acc: 0.3083 - val_loss: 1.0985 - val_acc: 0.3389

Epoch 00084: saving model to model2_3_1_chkp\weights.84-1.10.hdf5
Epoch 85/100
